In [1]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

In [3]:
import numpy as np
import pandas as pd                              # 결측치 체크
import sys
from tensorflow.keras import utils               # utils.to_categorical
from tensorflow.keras.models import Sequential   # 모델 만들기
from tensorflow.keras.layers import Dense, Input
from matplotlib import pyplot as plt

# 로지스틱 회귀분석(이진분류)
## 1. 데이터셋 생성 & 전처리 

In [6]:
df = pd.read_csv('data/pima-indians-diabetes.csv', 
             comment='#',
             header=None)
df.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [12]:
# 타겟 변수의 균형 
df.iloc[:,-1].value_counts() #df.loc[8]

0    500
1    268
Name: 8, dtype: int64

In [13]:
df.head()
# X_data=df.iloc[:,-1]

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
# csv 파일을 numpy배열로 바로 읽어오기 : np.loadtxt(결측치가 없는 경우), np.genfromtxt(결측치가 있을 경우)
dataset = np.loadtxt('data/pima-indians-diabetes.csv',
                     encoding='utf-8',
                     delimiter=',')
dataset.shape  # 독립변수 8개, 종속변수 1개

(768, 9)

In [21]:
# 데이터 분할 : 학습데이터셋(모델학습용) + 시험데이터셋(모델평가용)
X_train = dataset[:650, :-1]
y_train = dataset[:650, -1]
X_test = dataset[650: , :-1]
y_test = dataset[650: , -1]
X_train.shape , y_train.shape, X_test.shape, y_test.shape

((650, 8), (650,), (118, 8), (118,))